## Library

In [ ]:
import pickle

import pandas as pd
import os 

import cv2
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
!pip install tensorflow==2.8
!apt install —allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 668.3 MB 16 kB/s 
     |████████████████████████████████| 1.4 MB 4.1 MB/s 
     |████████████████████████████████| 5.8 MB 76.0 MB/s 
     |████████████████████████████████| 462 kB 75.6 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.1
    Uninstalling tensorflow-2.9.1:
      Successfully uninstalled tensorflow-2.9.1
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package —allow-change-held-packages


## Train & Test set 

In [ ]:
x_train = np.load('./x_train.npy')
y_train = np.load('./y_train.npy')
x_val = np.load('./x_val.npy')
y_val = np.load('./y_val.npy')
x_test = np.load('./x_test.npy')
y_test = np.load('./y_test.npy')

In [ ]:
print(x_train.shape,y_train.shape,x_val.shape,y_val.shape,x_test.shape,y_test.shape)

(15000, 224, 224, 3) (15000,) (1814, 224, 224, 3) (1814,) (1821, 224, 224, 3) (1821,)


## Resnet50

In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = 'imagenet') 

In [ ]:
base_model.output_shape

(None, 7, 7, 2048)

In [ ]:
base_model.trainable = True

In [ ]:
for layer in base_model.layers[:-10]: 
  layer.trainable = False				

for layer in base_model.layers[-100:]:	
  print(layer.name, layer.trainable)

conv3_block4_2_bn False
conv3_block4_2_relu False
conv3_block4_3_conv False
conv3_block4_3_bn False
conv3_block4_add False
conv3_block4_out False
conv4_block1_1_conv False
conv4_block1_1_bn False
conv4_block1_1_relu False
conv4_block1_2_conv False
conv4_block1_2_bn False
conv4_block1_2_relu False
conv4_block1_0_conv False
conv4_block1_3_conv False
conv4_block1_0_bn False
conv4_block1_3_bn False
conv4_block1_add False
conv4_block1_out False
conv4_block2_1_conv False
conv4_block2_1_bn False
conv4_block2_1_relu False
conv4_block2_2_conv False
conv4_block2_2_bn False
conv4_block2_2_relu False
conv4_block2_3_conv False
conv4_block2_3_bn False
conv4_block2_add False
conv4_block2_out False
conv4_block3_1_conv False
conv4_block3_1_bn False
conv4_block3_1_relu False
conv4_block3_2_conv False
conv4_block3_2_bn False
conv4_block3_2_relu False
conv4_block3_3_conv False
conv4_block3_3_bn False
conv4_block3_add False
conv4_block3_out False
conv4_block4_1_conv False
conv4_block4_1_bn False
conv4_bloc

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


Early stopping


In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5) 

In [ ]:
checkpoint_path = "./cp-{epoch:01d}.ckpt"  
checkpoint_dir = os.path.dirname(checkpoint_path)

Model check point

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss', 
    save_best_only=True, 
    save_weights_only=True, 
    )

Reduce learning rate

In [ ]:
callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1) 

In [ ]:
model_name = 'version.1'
current_time = datetime.datetime.now().strftime("%Y%m%d") 
current_time

In [ ]:
callback_list = [cp_callback,early,callback,tb_callback] 

In [ ]:
from tabnanny import verbose


model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# 모델 fitting
model.fit(x_train, y_train, epochs = 100, validation_data=(x_val,y_val), batch_size=125,callbacks=callback_list,verbose=1)

Epoch 1/100
120/120 [==============================] - 42s 310ms/step - loss: 2.0615 - accuracy: 0.3666 - val_loss: 0.9745 - val_accuracy: 0.5628 - lr: 0.0010
Epoch 2/100
120/120 [==============================] - 33s 275ms/step - loss: 1.0183 - accuracy: 0.5656 - val_loss: 0.8473 - val_accuracy: 0.6086 - lr: 0.0010
Epoch 3/100
120/120 [==============================] - 33s 276ms/step - loss: 0.8988 - accuracy: 0.6255 - val_loss: 0.7795 - val_accuracy: 0.6764 - lr: 0.0010
Epoch 4/100
120/120 [==============================] - 33s 280ms/step - loss: 0.7877 - accuracy: 0.6805 - val_loss: 0.7486 - val_accuracy: 0.6781 - lr: 0.0010
Epoch 5/100
120/120 [==============================] - 31s 258ms/step - loss: 0.7100 - accuracy: 0.7223 - val_loss: 0.7697 - val_accuracy: 0.6566 - lr: 0.0010
Epoch 6/100
120/120 [==============================] - 33s 275ms/step - loss: 0.6066 - accuracy: 0.7580 - val_loss: 0.7246 - val_accuracy: 0.6896 - lr: 2.0000e-04
Epoch 7/100
120/120 [=====================

In [ ]:
model.evaluate(x_test,y_test)

57/57 [==============================] - 3s 56ms/step - loss: 0.7310 - accuracy: 0.7013


[0.7310108542442322, 0.7012630701065063]

## Tensorboard 시각화

In [ ]:
%load_ext tensorboard

In [ ]:
import datetime

In [ ]:
#텐서보드
log_dir = "/logs/fit/" + current_time + model_name
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                      histogram_freq=1) 

In [ ]:
callback_list2 = [cp_callback,early,callback]

In [ ]:
## fine-tune 
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 100, validation_data=(x_val,y_val), batch_size=125,callbacks=callback_list2)